In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [24]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [25]:
# Store Part I results into DataFrame
# Load the csv exported in Part I to a DataFrame

In [26]:
city_data = '../WeatherPy/city_data_adam.csv'
city_data_df = pd.read_csv(city_data)
city_data_df = city_data_df.dropna()
city_data_df

,city_id,city,country,date_time,latitude,longitude,temperature (F),humidity (%),cloudiness (%),wind speed (mph)
0,3833367,Ushuaia,AR,1597379478,-54.80,-68.30,30.20,92,75,5.82
1,2264557,Ponta do Sol,PT,1597379675,32.67,-17.10,71.60,68,40,11.41
2,3691582,Talara,PE,1597379656,-4.58,-81.27,63.45,87,64,14.47
3,3672290,Piedras,CO,1597380065,4.54,-74.88,77.00,74,100,0.60
4,964712,Plettenberg Bay,ZA,1597380065,-34.05,23.37,49.98,74,2,4.72
...,...,...,...,...,...,...,...,...,...,...
554,5526337,Midland,US,1597380140,32.00,-102.08,91.40,25,7,9.17
555,5861897,Fairbanks,US,1597379645,64.84,-147.72,62.60,63,40,2.71
556,1563281,Tuy Hoa,VN,1597379701,13.08,109.30,90.93,55,100,3.58
557,5864145,Homer,US,1597380140,59.64,-151.55,57.20,82,1,13.87


In [27]:
# Humidity Heatmap

# Use the Lat and Lng as locations and Humidity as the weight.
# Add Heatmap layer to map.

locations = city_data_df[["latitude", "longitude"]].astype(float)
humidity = city_data_df["humidity (%)"].astype(float)
#printing map and defining weights using Humidity
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# Create new DataFrame fitting weather criteria
# Narrow down the cities to fit weather conditions.
# Drop any rows will null values.

In [29]:
vacay_df = city_data_df.loc[city_data_df["wind speed (mph)"] < 10]
vacay_df = vacay_df.loc[(vacay_df["temperature (F)"] < 80) | (vacay_df["temperature (F)"] > 70) ]
vacay_df = vacay_df.loc[vacay_df["cloudiness (%)"] == 0]
vacay_df = vacay_df.loc[vacay_df["humidity (%)"] < 30]
if len(vacay_df) > 10:
    vacay_df = vacay_df[:-(len(vacay_df)-10)]
vacay_df

,city_id,city,country,date_time,latitude,longitude,temperature (F),humidity (%),cloudiness (%),wind speed (mph)
20,6355222,Yulara,AU,1597379669,-25.24,130.99,69.80,26,0,5.82
193,1267776,Kargil,IN,1597380091,34.57,76.10,65.84,28,0,2.80
217,131962,Harsīn,IR,1597380094,34.27,47.59,78.80,16,0,3.67
252,2077895,Alice Springs,AU,1597380040,-23.70,133.88,71.60,24,0,4.70
297,6533368,Moranbah,AU,1597380089,-22.00,148.05,84.54,23,0,9.44
370,1170706,Mastung,PK,1597380117,29.80,66.84,88.14,19,0,2.46
417,95446,Erbil,IQ,1597380122,36.19,44.01,82.40,28,0,3.36
517,1519938,Karatau,KZ,1597380135,43.17,70.47,81.34,21,0,6.11


In [30]:
# Hotel Map
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

In [31]:
hotel_df = vacay_df
hotel_df["hotel_name"] = ""
hotel_df

,city_id,city,country,date_time,latitude,longitude,temperature (F),humidity (%),cloudiness (%),wind speed (mph),hotel_name
20,6355222,Yulara,AU,1597379669,-25.24,130.99,69.80,26,0,5.82,
193,1267776,Kargil,IN,1597380091,34.57,76.10,65.84,28,0,2.80,
217,131962,Harsīn,IR,1597380094,34.27,47.59,78.80,16,0,3.67,
252,2077895,Alice Springs,AU,1597380040,-23.70,133.88,71.60,24,0,4.70,
297,6533368,Moranbah,AU,1597380089,-22.00,148.05,84.54,23,0,9.44,
370,1170706,Mastung,PK,1597380117,29.80,66.84,88.14,19,0,2.46,
417,95446,Erbil,IQ,1597380122,36.19,44.01,82.40,28,0,3.36,
517,1519938,Karatau,KZ,1597380135,43.17,70.47,81.34,21,0,6.11,


In [32]:
hotels = []
for i, row in hotel_df.iterrows():
    lat = row["latitude"] 
    lng = row["longitude"] 
    target_radius = 5000

    # set up a parameters dictionary
    params = {
        "location": f"{lat}, {lng}",
        "type": "lodging",
        "key": g_key,
        "radius": target_radius,
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    # row["Hotel Name"] = response["results"][0]["name"]
    try:
        hotels.append(response["results"][0]["name"])
    except(IndexError):
        hotels.append("No Hotels in Target Radius")
hotel_df["hotel_name"] = hotels
hotel_df

,city_id,city,country,date_time,latitude,longitude,temperature (F),humidity (%),cloudiness (%),wind speed (mph),hotel_name
20,6355222,Yulara,AU,1597379669,-25.24,130.99,69.80,26,0,5.82,Desert Gardens Hotel - Ayers Rock Resort
193,1267776,Kargil,IN,1597380091,34.57,76.10,65.84,28,0,2.80,Caravan Sarai
217,131962,Harsīn,IR,1597380094,34.27,47.59,78.80,16,0,3.67,ایزوگام اصل دلیجان و شرق
252,2077895,Alice Springs,AU,1597380040,-23.70,133.88,71.60,24,0,4.70,Desert Palms Alice Springs
297,6533368,Moranbah,AU,1597380089,-22.00,148.05,84.54,23,0,9.44,Direct Hotels - Monterey Moranbah
370,1170706,Mastung,PK,1597380117,29.80,66.84,88.14,19,0,2.46,Syed House
417,95446,Erbil,IQ,1597380122,36.19,44.01,82.40,28,0,3.36,Erbil International Hotel
517,1519938,Karatau,KZ,1597380135,43.17,70.47,81.34,21,0,6.11,Gor.bol'nitsa


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [34]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))